# Some computations with the orbits

Here, we compute the orbits of a small variety of walks, also their rational invariants and decouplings. Most computations use the implementation in the python class `WalkOrbit2D`, which computes a finite orbit, invariants and decoupling.

## *Overview of this worksheet*

To make this portable, I have included the source code at the top.
You do not need to read it, the worksheet really starts afterwards.

We compute here several models with a finite orbits :
- The weighted models ($\mathcal{G}_{\lambda}$)
- Another extension of the Gessel models $\mathcal{H}_n$ for $n=2,3$

In [102]:
"""
    This code performs basic operation on the orbit of a walk, such
    as computing it, invariants of the walk and decoupling of rational
    fractions.
"""

from sage.arith.misc import factor
from sage.matrix.special import companion_matrix
from sage.misc.misc_c import prod
from sage.rings.fraction_field import FractionField
from sage.rings.polynomial.polynomial_ring_constructor import PolynomialRing
from sage.rings.rational_field import QQ
from sage.rings.power_series_ring import PowerSeriesRing
from sage.modules.free_module_element import free_module_element as vector
from sage.rings.big_oh import O
from sage.rings.ideal import Ideal
from sage.rings.qqbar import QQbar
from sage.rings.integer_ring import Z as ZZ
from sage.rings.real_mpfr import RR
from sage.misc.functional import sqrt
from sage.rings.polynomial.polynomial_ring import polygen, polygens


class WalkOrbit2D:
    """ This class defines the orbit, and offers several methods to compute with
    it.

    EXAMPLES::

        sage: krew = WalkOrbit2D([(-1,0),(0,-1),(1,1)])
        sage: krew.rational_invariant()
        ((-x^3*t - x + t)/(x^2*t), (-y^3*t - y + t)/(y^2*t))

        sage: gess3 = WalkOrbit2D("1/x/y + y + x + x/y + x^2*y")
        sage: gess3.rational_invariant()
        ((-x^6*t^2 - x^4*t^2 - x^4*t - x^3*t^2 + x^2*t^2 + x^3 + x^2*t + t^2)/(x^5*t^2 + 2*x^3*t^2 + x*t^2),
             (-y^4*t + y^3 + y*t + t)/(y^2*t))

        sage: oo = WalkOrbit2D("1/x/y + y + x + x/y + x^2*y + p*x", wnames=['p'])
        sage: oo.does_decouple('X*Y')
        True

    """

    def __init__(self, model, wnames=[], autocompute=True):
        # the rings in which we work
        self.R = PolynomialRing(QQ, names=['x', 'y', 't', 'X', 'Y'] + wnames)
        self.Rfrac = FractionField(self.R)
        self.rgd = self.R.gens_dict()
        (x, y, t, X, Y,) = self.R._first_ngens(5)

        self.Cinv = FractionField(PolynomialRing(QQ, [t] + [self.rgd[u] for u in wnames]))

        # model can be given either by a list of pairs, or by a laurent
        # polynomial in x and y
        if isinstance(model, str):
            self.S = self.Rfrac(model)
        else:
            self.S = self.Rfrac(sum(x**i * y**j for i, j in model))

        # The support of the model
        mx = self.S.denominator().degree(x)
        my = self.S.denominator().degree(y)
        self.support = [(u[0]-mx,u[1]-my) for u in self.S.numerator().dict()]

        self.mk = (1-t*self.S).numerator()
        self.K = self.mk.subs(x=X,y=Y)
        self.dx = self.K.degree(X)
        self.dy = self.K.degree(Y)

        # now, some tests

        self.compute()

    def __repr__(self):

        # TODO adapt depending if the computation time exceeded
        sz = "size {s}".format(s=self.size)

        string = "Orbit of a 2D walk of degree {dx}x{dy} and of {size}".format(
            dx=self.dx,
            dy=self.dy,
            size=sz)

        return string

    def _sqrf(self, P, Q, s):
        """ Extracts the nonconstant factors of the polynomial P regarding
        indeterminate s which are coprimes to Q.
        """

        # factorization = factor(P)
        # acc = 1
        # for f, _  in factorization:
        #     if self.R(f).degree(s) >= 1 and Q.gcd(self.R(f)) == 1:
        #         #if acc != 1:
        #         #    print("Warning : a wild multiple factor appears !")
        #         acc *= self.R(f)
        # return acc
        p = P / P.gcd(P.derivative(s))
        return self.R(p / Q)

    def _sumRoots(self, H, P, s):
        """ If P is a polynomial, this function
        evaluates the sum of H over all of its roots.
        """
        
        U = polygen(self.Rfrac, 'U')
        p = P.subs({s:U}).polynomial(U)
        S = self.Rfrac[U].quotient(p)
        u = S.gen()
        h = S(H.subs({s:u}))
        
        return h.trace()

    def compute(self):
        """ This functions builds the orbit of the walk.

        If the orbit is finite, this function returns the information
        of the orbit as two pairs of polynomials, each one related
        to the line levels of the walk.

        The first pair
            ([P0(t,x,X), ..., Pn(t,x,X)],
             [Q0(t,x,Y), ..., Qm(t,x,Y)])
        encodes the x-level lines in the following way.

        The points of the orbit at distance 2i-1
        or 2i from the origin are the pairs (u,v) which
        are roots of S(u,v) - S(x,y) and such that u is a
        root of Pi(t,x,u).
        Conversely, the points of the orbit at
        distance 2i or 2i+1 from the origin are the pairs (u,v)
        which are roots of S(u,v) - S(x,y) and such that
        v is a root of Qi(t,x,v).

        The second pair encodes the y-level lines
        in a similar fashion, with the roles of Pi and Qi swapped.

        This function doesn’t currently terminate if the orbit is infinite.
        """
        (x, y, t, X, Y,) = self.R._first_ngens(5)

        # Construction des x-lignes de niveaux
        P = X-x; Psx = [X-x];
        Q = self.R(1); Qsx = [];
        dm = 1

        while True:
            Q = self._sqrf(self.K.resultant(P,X),Q,Y)
            dm = max(dm, Q.degree(Y))
            if Q.degree(Y) == 0: break
            Qsx.append(Q)

            P = self._sqrf(self.K.resultant(Q,Y),P,X)
            dm = max(dm, P.degree(X))
            if P.degree(X) == 0: break
            Psx.append(P)

        # Construction des y-lignes de niveaux
        P = self.R(1); Psy = [];
        Q = Y-y; Qsy = [Y-y];

        while True:
            P = self._sqrf(self.K.resultant(Q,Y),P,X)
            if P.degree(X) == 0: break
            Psy.append(P)

            Q = self._sqrf(self.K.resultant(P,X),Q,Y)
            if Q.degree(Y) == 0: break
            Qsy.append(Q)

        self.lines = Psx, Qsx, Psy, Qsy
        self.size = prod(Psx).degree(X) * self.K.degree(Y)

        self.nx = len(Psx) + len(Qsx) - 1
        self.ny = len(Psy) + len(Qsy) - 1

        # computes the degrees of each line
        d1 = [self.Rfrac(self._deg_x(i)) for i in range(self.nx)]
        for i in range(1, len(d1)):
            d1[i] -= d1[i-1]

        d2 = [self.Rfrac(self._deg_y(i)) for i in range(self.ny)]
        for i in range(1, len(d2)):
            d2[i] -= d2[i-1]

        self.degrees = d1, d2

    def _sum_over_left(self, h, p):
        X = self.rgd['X']
        Y = self.rgd['Y']
        a = self._sumRoots(h, self.K, Y)
        b = self._sumRoots(a, p, X)
        return b

    def _sum_over_right(self, h, p):
        X = self.rgd['X']
        Y = self.rgd['Y']
        a = self._sumRoots(h, self.K, X)
        b = self._sumRoots(a, p, Y)
        return b

    def _sum_x(self, h, i):
        # returns the sum of h over x-lines i and i-1

        if i % 2 == 0:
            return self._sum_over_left(h, self.lines[0][i//2])
        else:
            return self._sum_over_right(h, self.lines[1][i//2])

    def _sum_y(self, h, i):
        # returns the sum of h over y-lines i and i-1

        if i % 2 == 0:
            return self._sum_over_right(h, self.lines[3][i//2])
        else:
            return self._sum_over_left(h, self.lines[2][i//2])

    def _deg_x(self, i):
        # returns the number of vertices at x distance i or i-1
        X = self.rgd['X']
        Y = self.rgd['Y']
        if i % 2 == 0:
            return self.lines[0][i//2].degree(X) * self.dy
        else:
            return self.lines[1][i//2].degree(Y) * self.dx

    def _deg_y(self, i):
        # returns the number of vertices at y distance i or i-1
        X = self.rgd['X']
        Y = self.rgd['Y']
        if i % 2 == 0:
            return self.lines[3][i//2].degree(Y) * self.dx
        else:
            return self.lines[2][i//2].degree(X) * self.dy

    def _inv_gens(self):
        X = self.rgd['X']
        px, _, py, _ = self.lines
        p = prod(px).polynomial(X).monic().coefficients()
        q = prod(py).polynomial(X).monic().coefficients()

        return [a for a in p if a not in self.Cinv], [b for b in q if b not in self.Cinv]

    def rational_invariant(self):
        """ Returns a pair of rational invariant that spans the field
        of rational invariants over Q(t). """
        a,b = self._inv_gens()
        return a[0], b[0]

    def decoupl(self, h):
        """ This functions computes the decoupling of a given rational fraction
        h in X, Y and t. It returns a tuple of fractions in x, y and t
        u(x,t), v(y,t) and r(x,y,t) such that h(x,y,1/S(x,y)) = u(x,1/S(x,y)) + v(y,1/S(x,y))
        iff h admits a decoupling in this model. In that case, h(x,y,t) = u(x,t) + v(y,t) + r(x,y,t) * K(x,y,t)
        """

        (x, y, t, X, Y,) = self.R._first_ngens(5)
        h = self.Rfrac(h)
        nx = self.nx
        ny = self.ny

        # on calcule les sommes de h sur chaque ligne de niveau
        ssx = [self._sum_x(h, j) for j in range(nx)]
        for i in range(1,nx):
            ssx[i] -= ssx[i-1]

        ssy = [self._sum_y(h, j) for j in range(ny)]
        for i in range(1,ny):
            ssy[i] -= ssy[i-1]

        # on calcule la somme en x
        d = self.degrees[0]
        sx = - sum(ssx) # on ajoute l’invariant à la partie en x
        sd = self.size
        for j in range(1, nx, 2):
            sd -= d[j-1] + (d[j-2] if j >= 3 else 0)
            sx += sd/d[j] * ssx[j] - sd/d[j-1] * ssx[j-1]

        # de même pour y
        d = self.degrees[1]
        sy = self.Rfrac(0)
        sd = self.size
        for j in range(1, ny, 2):
            sd -= d[j-1] + (d[j-2] if j >= 3 else 0)
            sy += sd/d[j] * ssy[j] - sd/d[j-1] * ssy[j-1]

        nh = self.Rfrac(h.subs({X:x,Y:y}))

        return (- sx / self.size,
                - sy / self.size,
                (nh + sx / self.size + sy / self.size) / self.mk)

    def does_decouple(self, h):
        """ This functions tests whether a given rational function
        in X, Y and t decouples in this model. """
        return self.obstruct_decoupl(h) == 0


    def obstruct_decoupl(self, h):
        """ This functions measures whether a given rational function
        in X, Y and t decouples in this model. """

        (x, y, t, X, Y,) = self.R._first_ngens(5)
        u, v, _ = self.decoupl(h)
        return (u+v-self.Rfrac(h)).subs({X:x,Y:y,t:1/self.S})


## The model $G_{\lambda}$ (with orbit $\mathcal{O}_{12}$):

In [103]:
glamb = WalkOrbit2D("1/x/y + y + lamb*x + x/y + x^2*y", wnames=['lamb']); glamb

Orbit of a 2D walk of degree 3x2 and of size 12

In [104]:
glamb.rational_invariant()

((-x^6*t^2 - x^3*t^2*lamb^2 - x^4*t^2 - x^4*t*lamb + x^2*t^2 + x^2*t*lamb + x^3 + t^2)/(x^5*t^2 + 2*x^3*t^2 + x*t^2),
 (-y^4*t + y^3 + y*t*lamb + t)/(y^2*t))

In this model, $xy$ does indeed decouple.

In [105]:
glamb.does_decouple('X*Y')

True

In [106]:
glamb.decoupl('X*Y')

((9*x^2*t*lamb - 3*t*lamb - 12*x)/(-12*x^2*t - 12*t),
 (-3*y*lamb - 12)/(12*y),
 (-144)/(144*x^2*y*t + 144*y*t))

## The model $\mathcal{H}_2$

In [111]:
oo = WalkOrbit2D("1/x*(1/y+1)+(y+1)*x^2"); oo

Orbit of a 2D walk of degree 3x2 and of size 18

Rational invariants :

In [112]:
i,j=oo.rational_invariant(); i, j

((-x^9*t^2 + 2*x^7*t + 3*x^6*t^2 - x^5 - x^4*t - x*t + t^2)/(x^3*t^2),
 (-y^6*t^3 - 3*y^5*t^3 - 6*y^4*t^3 - 11*y^3*t^3 - 12*y^2*t^3 - y^4 - 6*y*t^3 - t^3)/(y^5*t^3 + 3*y^4*t^3 + 3*y^3*t^3 + y^2*t^3))

In [113]:
oo.does_decouple("X^2*Y")

True

In [114]:
oo.decoupl("X^2*Y")

((6*x - 18*t)/(18*x*t),
 (-12*y + 6)/(-18*y*t - 18*t),
 (-324)/(324*x*y*t + 324*x*t))

In [115]:
oo.does_decouple("X^3*Y")

True

In [116]:
oo.decoupl("X^3*Y")

((-18*x^3*t + 18*x)/(18*t), (-18*y - 18)/(18*y), 324/(-324*y*t))

In [117]:
oo.does_decouple("X^6*Y^2")

True

In [118]:
oo.decoupl("X^6*Y^2")

((12*x^9*t^2 - 24*x^7*t + 18*x^6*t^2 + 12*x^5 - 42*x^4*t - 18*x^3*t^2 - 6*x*t + 6*t^2)/(18*x^3*t^2),
 (2324522934*y^6*t^3 + 20920706406*y^5*t^3 + 55788550416*y^4*t^3 + 60437596284*y^3*t^3 + 20920706406*y^2*t^3 + 2324522934*y^4 - 6973568802*y*t^3 - 4649045868*t^3)/(6973568802*y^5*t^3 + 20920706406*y^4*t^3 + 20920706406*y^3*t^3 + 6973568802*y^2*t^3),
 (125524238436*x^6*y^5*t^2 + 251048476872*x^6*y^4*t^2 + 41841412812*x^6*y^3*t^2 - 167365651248*x^6*y^2*t^2 + 125524238436*x^4*y^4*t - 83682825624*x^6*y*t^2 - 125524238436*x^3*y^4*t^2 + 292889889684*x^4*y^3*t - 376572715308*x^3*y^3*t^2 + 251048476872*x^4*y^2*t - 292889889684*x^3*y^2*t^2 + 83682825624*x^4*y*t + 41841412812*x^3*y*t^2 - 41841412812*y^4*t^2 + 41841412812*x^2*y^3 + 41841412812*x*y^3*t + 83682825624*x^3*t^2 - 83682825624*y^3*t^2 + 41841412812*x*y^2*t - 41841412812*y^2*t^2)/(-125524238436*x^3*y^5*t^3 - 376572715308*x^3*y^4*t^3 - 376572715308*x^3*y^3*t^3 - 125524238436*x^3*y^2*t^3))

In [119]:
oo.does_decouple("X^9*Y^3")

True

In [120]:
oo.decoupl("X^9*Y^3")

((-18*x^12*t^3 + 54*x^10*t^2 + 27*x^9*t^3 - 54*x^8*t - 81*x^6*t^3 + 18*x^6 - 27*x^5*t + 81*x^4*t^2 + 81*x^3*t^3 + 27*x*t^2 - 27*t^3)/(18*x^3*t^3),
 (-10460353203*y^7*t^3 - 69735688020*y^6*t^3 - 177826004451*y^5*t^3 - 230127770466*y^4*t^3 - 170852435649*y^3*t^3 - 10460353203*y^5 - 83682825624*y^2*t^3 - 31381059609*y*t^3 - 6973568802*t^3)/(6973568802*y^6*t^3 + 20920706406*y^5*t^3 + 20920706406*y^4*t^3 + 6973568802*y^3*t^3),
 (125524238436*x^9*y^7*t^2 + 251048476872*x^9*y^6*t^2 + 125524238436*x^9*y^5*t^2 + 125524238436*x^9*y^4*t^2 + 125524238436*x^7*y^6*t + 251048476872*x^9*y^3*t^2 - 125524238436*x^6*y^6*t^2 + 125524238436*x^7*y^5*t + 125524238436*x^9*y^2*t^2 - 251048476872*x^6*y^5*t^2 - 376572715308*x^7*y^4*t - 313810596090*x^6*y^4*t^2 - 627621192180*x^7*y^3*t - 502096953744*x^6*y^3*t^2 + 125524238436*x^5*y^5 - 251048476872*x^7*y^2*t - 251048476872*x^4*y^5*t - 439334834526*x^6*y^2*t^2 + 125524238436*x^3*y^5*t^2 + 376572715308*x^5*y^4 - 251048476872*x^4*y^4*t - 125524238436*x^6*y*t^2 + 81

## The model $\mathcal{H}_3$

In [121]:
oo = WalkOrbit2D("1/x*(1/y+1)+(y+1)*x^3");oo

Orbit of a 2D walk of degree 4x2 and of size 32

In [122]:
i,j=oo.rational_invariant(); i, j

((-x^16*t^3 + 3*x^13*t^2 + 4*x^12*t^3 - 3*x^10*t - 5*x^9*t^2 - 6*x^8*t^3 + x^7 + x^6*t + x^5*t^2 + x*t^2 - t^3)/(x^4*t^3),
 (y^8*t^4 + 4*y^7*t^4 + 12*y^6*t^4 + 32*y^5*t^4 + 54*y^4*t^4 + 52*y^3*t^4 + y^6 + 28*y^2*t^4 + 8*y*t^4 + t^4)/(y^7*t^4 + 4*y^6*t^4 + 6*y^5*t^4 + 4*y^4*t^4 + y^3*t^4))

In [123]:
oo.does_decouple("X^3*Y")

True

In [124]:
oo.decoupl("X^3*Y")

((8*x - 32*t)/(32*x*t),
 (24*y - 8)/(32*y*t + 32*t),
 1024/(-1024*x*y*t - 1024*x*t))

In [125]:
oo.does_decouple("X^4*Y")

True

In [126]:
oo.decoupl("X^4*Y")

((-32*x^4*t + 32*x)/(32*t), (-32*y - 32)/(32*y), 1024/(-1024*y*t))

In [127]:
oo.does_decouple("X^8*Y^2")

True

In [128]:
oo.decoupl("X^8*Y^2")

((32*x^8*t^2 - 64*x^5*t + 32*x^2 - 64*x*t)/(32*t^2),
 (32*y^2 - 32)/(32*y^2),
 (1024*x^4*y^2*t - 1024*x^4*y*t + 1024*x*y - 1024*y*t + 1024*t)/(-1024*y^2*t^2))

In [129]:
oo.does_decouple("X^12*Y^3")

True

In [130]:
oo.decoupl("X^12*Y^3")

((-20*x^16*t^3 + 60*x^13*t^2 - 48*x^12*t^3 - 60*x^10*t + 156*x^9*t^2 + 72*x^8*t^3 + 20*x^7 - 108*x^6*t + 84*x^5*t^2 - 48*x^4*t^3 - 12*x*t^2 + 12*t^3)/(32*x^4*t^3),
 (41204205843510149815894189071372/542101086242752217003726400434970855712890625*y^8*t^4 + 219755764498720799018102341713984/542101086242752217003726400434970855712890625*y^7*t^4 + 714206234620842596808832610570448/542101086242752217003726400434970855712890625*y^6*t^4 + 329633646748081198527153512570976/108420217248550443400745280086994171142578125*y^5*t^4 + 466980999559781697913467476142216/108420217248550443400745280086994171142578125*y^4*t^4 + 1758046115989766392144818733711872/542101086242752217003726400434970855712890625*y^3*t^4 + 41204205843510149815894189071372/542101086242752217003726400434970855712890625*y^6 + 494450470122121797790730268856464/542101086242752217003726400434970855712890625*y^2*t^4 - 109877882249360399509051170856992/542101086242752217003726400434970855712890625*y*t^4 - 1373473528117004993863139635712